In [ ]:
#This Notebook will be the one that generates map according to the activity you want to do
#Different activities are in diffrents cells
#Cards are generated in the File Output/Maps/. 
# You need to open them with a webbrowser. You can either install the 'Open with browser VSCODE extension'ArithmeticError' 
# or use Jupyterlab.

In [ ]:
import folium
import pandas as pd 
from folium.plugins import MarkerCluster
import webbrowser
from datetime import datetime

In [4]:
#Cinema: This code will plot a map giving you all the movie stream the 24 december 2023 in Paris between
# two hours that are MinHour and Maxhour

def HourConversion(string):                  #Converts the date in the Dataframe in a Datetime object
    return datetime.strptime(string, '%H:%M') 

def MovieMapping(data,MinHour:'HHhMM',MaxHour:'HHhMM'):      #Plots the map
    
    HeureDebut = datetime.strptime(MinHour, "%Hh%M")
    HeureFin = datetime.strptime(MaxHour, "%Hh%M")
    
    
    data['time'] = data['heure'].apply(lambda x: HourConversion(x))
    AdjustedData = data[data['time']>= HeureDebut]
    AdjustedData = AdjustedData[AdjustedData['time'] <= HeureFin]
    
    MovieMap = folium.Map(location=[48.8566, 2.3522], zoom_start=12)
    GeoVisited = {}

    for index, row in AdjustedData.iterrows():
        lat = row['geo'].split(',')[0]
        lon = row['geo'].split(',')[1]
    
        content = "<h4 style='color:black;'>" + row['nom'] + "</h4>" \
                "<p style='font-size:16px;'>" + row['etablissement'] + "</p>"\
                "<p style='font-size:16px;'>" + row['heure'] + "</p>"
        
        if (lat,lon) not in GeoVisited.keys():
            GeoVisited[(lat,lon)] = MarkerCluster().add_to(MovieMap)
            folium.Marker(location=(lat,lon), popup = content, max_width=500).add_to(GeoVisited[(lat,lon)])
        else:
            folium.Marker(location=(lat,lon), popup = content, max_width=500).add_to(GeoVisited[(lat,lon)])
    MovieMap.save("/home/onyxia/work/Maps_cultural_life_Paris/Outputs/Maps/MovieMap.html")
    

#Example: Display all the projections that begin between 17h30 and 20h00 on the 28 th of December (ref Date for the Dataset)
    
program = pd.read_csv('/home/onyxia/work/Maps_cultural_life_Paris/Outputs/DataSets/DataCinema.csv')
MovieMapping(program,'17h30','20h00')       # We find all the movie streamed between 17h30 and 20h00
                                            # You can change the times as you want



In [7]:
#Theater: This code will plot a map giving you all theater pieces played on a precise date in Paris.

def TheaterMap(data:'Pandas DataFrame',date:'YYYY-MM-DD'):

    user_date = datetime.strptime(date, '%Y-%m-%d')
    data['date début'] = pd.to_datetime(data['date début'])
    data['date fin'] = pd.to_datetime(data['date fin'])
    filtered_data = data[(data['date début'] <= user_date) & (user_date <= data['date fin'])]

    paris_coordinates = [48.8566, 2.3522]
    my_map = folium.Map(location=paris_coordinates, zoom_start=12)

    for index, row in filtered_data.iterrows():
        establishment_name = row['etablissement']
        address = row['adresse']
        show_name = row['nom']
        average_price = row['prix moyen']

        coordinates = [float(coord.strip('()')) for coord in row['Coordonnees'].split(',')]
        popup_text = f"<b>{establishment_name}</b><br>Adresse: {address}<br>Pièce: {show_name}<br>Prix moyen: {average_price} €"
        folium.Marker(location=coordinates, popup=popup_text).add_to(my_map)
    
    my_map.save("/home/onyxia/work/Maps_cultural_life_Paris/Outputs/Maps/Theatermap.html")

#Example: Display all theater pieces played the 12th january 2024 in Paris
DataTheater = pd.read_csv('/home/onyxia/work/Maps_cultural_life_Paris/Outputs/DataSets/DataTheatre_base_finale.csv', sep=';')
TheaterMap(DataTheater,'2024-01-12' )

In [ ]:
#Concert: 